<a href="https://colab.research.google.com/github/Shushukang/NLP/blob/main/%EC%8B%A4%EC%8A%B514.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이미지 유사도 측정

## SentenceTransformer 이용

In [ ]:
# 필요한 라이브러리 설치
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 3.1 MB/s eta 0:00:00


In [ ]:
### 설치한 라이브러리로부터 필요한 함수 임폴트
from sentence_transformers import SentenceTransformer
from PIL import Image

In [ ]:
### 사전 학습된 모델 다운로드
model_name="clip-ViT-B-32"
model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

0_CLIPModel/special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

...

0_CLIPModel/preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

0_CLIPModel/config.json:   0%|          | 0.00/4.03k [00:00<?, ?B/s]

0_CLIPModel/vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
### 이미지 → 임베딩 벡터 추출 함수 설정

# 가장 빠르게 임베딩 벡터를 추출할 수 있는 방법

def extract_embedding(path):
  # 이미지 불러오기
  img = Image.open(path)
  # model.encode(img) 호출
  embedding = model.encode(img)
  return embedding

In [ ]:
### 이미지 → 임베딩 벡터 추출 함수 실행

# 필요한 라이브러리 임폴트
import numpy as np


# 이미지 경로 설정
file_path = ['/content/drive/MyDrive/딥러닝/cat1.jpg',
             '/content/drive/MyDrive/딥러닝/cat2.png',
             '/content/drive/MyDrive/딥러닝/dog1.jpg',
             '/content/drive/MyDrive/딥러닝/dog2.jpg']

# for문 + extract_embedding 함수 실행
embedding_list = []
for path in file_path:
  # 1개의 이미지 처리
  embedding = extract_embedding(path)
  # 1개의 이미지 처리 결과 저장
  embedding_list.append(embedding)

# 추출된 임베딩 벡터의 리스트 → 넘파이 배열로 변환
embedding_list_arr = np.array(embedding_list)

# 결과 확인하기
print(f'넘파이 배열로 변환된 임베딩 벡터의 배열 모양 : {embedding_list_arr.shape}')

넘파이 배열로 변환된 임베딩 벡터의 배열 모양 : (4, 512)


In [ ]:
### 코사인 유사도 측정

# 필요한 함수 임폴트
from sklearn.metrics.pairwise import cosine_similarity

# cosine_similarity(X, Y) 함수 호출
sim = cosine_similarity(embedding_list_arr, embedding_list_arr)
# 결과값은 2차원 배열로 도출됨

# 결과 확인하기
print(sim)

[[1.0000002  0.86011684 0.7399045  0.604113  ]
 [0.86011684 0.9999999  0.77550316 0.6457542 ]
 [0.7399045  0.77550316 0.99999964 0.74970865]
 [0.604113   0.6457542  0.74970865 0.9999999 ]]


## ViTransformer 이용

In [ ]:
### 임베딩 행렬 추출 함수 정의
# 필요시 이대로 가져다가 사용하면 됨
# 분류 모델이 아닌 그냥 ViT 모델

# 필요한 함수 임폴트
from transformers import ViTImageProcessor, ViTModel

# pretrained 모델을 이용한 전처리 모델 생성
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')


# pretrained 모델을 이용한 ViT 모델 객체 생성
model = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')


# 임베딩 추출 함수 정의
def feature_extract(file_path):
    # 이미지 읽기 + conver('RGB')는 코드를 조금 더 안전하게 하기 위해 추가
    img = Image.open(file_path).convert("RGB")
    # 입력 데이터 생성
    inputs = processor(images=img, return_tensors="pt")
    # ViTModel 모델 실행
    outputs = model(**inputs)
    # cls token 추출 : 197개의 모든 임베딩 벡터의 토큰이 추출됨 → 0번을 지정하여 CLS 토큰만 추출 → 결과물 넘파이 배열로 변환
    cls = outputs.last_hidden_state[:,0,:]
    cls_arr = cls.detach().numpy()


    return cls_arr

In [ ]:
### 이미지 → 임베딩 벡터 추출 함수 실행

# 이미지 경로 설정
file_path = ['/content/drive/MyDrive/딥러닝/cat1.jpg',
             '/content/drive/MyDrive/딥러닝/cat2.png',
             '/content/drive/MyDrive/딥러닝/dog1.jpg',
             '/content/drive/MyDrive/딥러닝/dog2.jpg']

# for문 + feature_extract 함수 실행
embedding_list = []
for path in file_path:
  # 1개의 이미지 처리
  embedding = feature_extract(path)
  # 1개의 이미지 처리 결과 저장
  embedding_list.append(embedding)

# 추출된 임베딩 벡터의 리스트 → 넘파이 배열로 변환
embedding_list_arr = np.array(embedding_list)

# 결과 확인하기
print(f'넘파이 배열로 변환된 임베딩 벡터의 배열 모양 : {embedding_list_arr.shape}')
# 결과가 2차원으로 도출이 되기 때문에 4개가 모여 3차원이 됨 → reshape 필요

넘파이 배열로 변환된 임베딩 벡터의 배열 모양 : (4, 1, 768)


In [ ]:
### 코사인 유사도 측정

# embedding_list_arr 모양 → 2차원 배열로 변환
embedding_list_arr = embedding_list_arr.reshape((4,768))

# cosine_similarity(X, Y) 함수 호출
sim = cosine_similarity(embedding_list_arr, embedding_list_arr)

# 결과 확인하기
print(sim)

[[1.0000001  0.5765541  0.15779383 0.13741112]
 [0.5765541  1.0000002  0.11201827 0.10642868]
 [0.15779383 0.11201827 0.9999999  0.82807356]
 [0.13741112 0.10642868 0.82807356 1.0000001 ]]
